In [1]:
import pandas as pd
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, Adafactor
import torch
from sklearn.model_selection import train_test_split

In [2]:
data_raw = pd.read_csv('data/news_summary.csv',encoding='latin-1')

In [3]:
data_raw.head()

,author,date,headlines,read_more,text,ctext
0,Chhavi Tyagi,"03 Aug 2017,Thursday",Daman & Diu revokes mandatory Rakshabandhan in...,http://www.hindustantimes.com/india-news/raksh...,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Daisy Mowke,"03 Aug 2017,Thursday",Malaika slams user who trolled her for 'divorc...,http://www.hindustantimes.com/bollywood/malaik...,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,Arshiya Chopra,"03 Aug 2017,Thursday",'Virgin' now corrected to 'Unmarried' in IGIMS...,http://www.hindustantimes.com/patna/bihar-igim...,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Sumedha Sehra,"03 Aug 2017,Thursday",Aaj aapne pakad liya: LeT man Dujana before be...,http://indiatoday.intoday.in/story/abu-dujana-...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Aarushi Maheshwari,"03 Aug 2017,Thursday",Hotel staff to get training to spot signs of s...,http://indiatoday.intoday.in/story/sex-traffic...,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...


In [63]:
data = pd.DataFrame()
data['news'] = data_raw['ctext']
data['summary'] = data_raw['text']

In [89]:
data.head()

,news,summary
0,The Daman and Diu administration on Wednesday ...,The Administration of Union Territory Daman an...
1,"From her special numbers to TV?appearances, Bo...",Malaika Arora slammed an Instagram user who tr...
2,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Hotels in Mumbai and other Indian cities are t...,Hotels in Maharashtra will train their staff t...


In [90]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4396 entries, 0 to 4513
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   news     4396 non-null   object
 1   summary  4396 non-null   object
dtypes: object(2)
memory usage: 103.0+ KB


In [75]:
data = data.dropna()

In [101]:
X = list(data.news)
y = list(data.summary)
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42)

In [95]:
model_name = 'google/pegasus-xsum'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)

In [272]:
#Gives a dictionary of tensors
input_enc_train = tokenizer.prepare_seq2seq_batch(X_train[0:5], truncation=True, padding='longest', return_tensors="pt")
target_enc_train = tokenizer.prepare_seq2seq_batch(y_train[0:5], truncation=True, padding='longest', return_tensors="pt")

In [246]:
class NewsSummaryDataset(torch.utils.data.Dataset):
    def __init__(self,input_encodings,target_encodings):
        self.input_encodings = input_encodings
        self.target_encodings = target_encodings
    
    def __getitem__(self,idx):
        data = {}
        data["encoder_input"]=self.input_encodings["input_ids"][idx]
        data["encoder_attention_masks"] = self.input_encodings["attention_mask"][idx]
        data["decoder_output"] = self.target_encodings["input_ids"][idx]
        decoder_input = self.target_encodings.input_ids[idx]
        decoder_input = torch.roll(decoder_input,1,-1)
        decoder_input[0] = torch.tensor(0)
        data["decoder_input"] = decoder_input
        return data
    
    def __len__(self):
        return self.target_encodings.input_ids.shape[0]


        

In [247]:
train_dataset = NewsSummaryDataset(input_enc_train,target_enc_train)

In [258]:
INPUT_BATCH_SIZE=16
train_loader = torch.utils.data.DataLoader(train_dataset,shuffle=True,batch_size=INPUT_BATCH_SIZE)

In [259]:
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)
model.train()

PegasusForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(96103, 1024, padding_idx=0)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(96103, 1024, padding_idx=0)
      (embed_positions): SinusoidalPositionalEmbedding(512, 1024)
      (layers): ModuleList(
        (0): EncoderLayer(
          (self_attn): Attention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        )


In [263]:
optim = Adafactor(model.parameters())

In [268]:
EPOCH = 2
for i in range(EPOCH):
    for batch in train_loader:
        optim.zero_grad()
        encoder_input = batch["encoder_input"].to(torch_device)
        encoder_attention_mask = batch["encoder_attention_masks"].to(torch_device)
        decoder_input = batch["decoder_input"].to(torch_device)
        decoder_output = batch["decoder_output"].to(torch_device)
        output = model(input_ids=encoder_input,
                     attention_mask=encoder_attention_mask,
                     decoder_input_ids=decoder_input,
                    labels=decoder_output)
        loss = output[0]
        loss.backward()
        optim.step()
model.eval()

/Users/gaurav/anaconda3/lib/python3.8/site-packages/transformers/optimization.py:506: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1607370249289/work/torch/csrc/utils/python_arg_parser.cpp:882.)
  exp_avg_sq_row.mul_(beta2t).add_(1.0 - beta2t, update.mean(dim=-1))


PegasusForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(96103, 1024, padding_idx=0)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(96103, 1024, padding_idx=0)
      (embed_positions): SinusoidalPositionalEmbedding(512, 1024)
      (layers): ModuleList(
        (0): EncoderLayer(
          (self_attn): Attention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        )


In [269]:
input_enc_test = tokenizer.prepare_seq2seq_batch(X_test[0:2], truncation=True, padding='longest', return_tensors="pt")
translated = model.generate(**input_enc_test)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)

In [270]:
tgt_text

['A woman has been arrested for trespassing at the White House.',
 'Priyanka Chopra has said that "I definitely think it is no one\'s business what a family decides to do." "I definitely think it is no one\'s business what a family decides to do," she added. "I definitely think it is no one\'s business what a family decides to do."']

In [ ]:
traget_enc_test = tokenizer.prepare_seq2seq_batch(y_test[0:2], truncation=True, padding='longest', return_tensors="pt")

In [273]:
y_test[0:2]

['A woman, who was arrested twice last week for trying to jump the White House fence, has been arrested for the third time for scaling a fence at the Treasury Building, next to the White House. The woman, who told officers that she wanted to speak to President Donald Trump, has been charged with unlawful entry and contempt of court.',
 "A tea stall in Gujarat's Vadnagar railway station where PM Narendra Modi used to sell tea during his childhood is set to become a tourist spot. Union Culture Minister Mahesh Sharma visited the place on Sunday and said the tea stall will be developed as a part of a ?100-crore project to develop Vadnagar and surrounding areas as tourist destinations. "]

In [16]:
from tqdm.auto import tqdm
from time import sleep

In [17]:
for i in tqdm(range(10)):
    sleep(3)